In [ ]:
!pip install -U scikit-learn
!pip install -U pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 25.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
google-colab 1.0.0 requires

In [ ]:
!pip install -U tqdm

In [2]:
import sklearn
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [3]:
# Montage du Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# Get the data
#data_clean = "/content/drive/MyDrive/LoLDrafter/data/Match_data11 _clean.csv"
data_clean_path = "/content/drive/MyDrive/LoLDrafter/data/match_data.csv"

columns = ['WIN'] + [f'ID_{k}' for k in range(1, 11)] + [f'ROLE_{k}' for k in range(1, 11)]
df = pd.read_csv(data_clean_path, sep=';', names=columns)
df = df.replace(' ', pd.NA).dropna(axis=0) # Filter out the rows that contain " "
df['WIN'] = df['WIN'].astype(int)

df.head()

,WIN,ID_1,ID_2,ID_3,ID_4,ID_5,ID_6,ID_7,ID_8,ID_9,ID_10,ROLE_1,ROLE_2,ROLE_3,ROLE_4,ROLE_5,ROLE_6,ROLE_7,ROLE_8,ROLE_9,ROLE_10
0,1,19.0,64.0,517.0,145.0,117.0,887.0,9.0,91.0,22.0,910.0,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY
1,1,54.0,64.0,134.0,81.0,35.0,55.0,9.0,48.0,498.0,117.0,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,MIDDLE,JUNGLE,TOP,BOTTOM,UTILITY
5,0,57.0,5.0,24.0,526.0,85.0,267.0,51.0,63.0,77.0,126.0,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY
6,1,117.0,2.0,113.0,498.0,112.0,147.0,104.0,32.0,122.0,876.0,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY
7,0,18.0,412.0,78.0,266.0,68.0,122.0,58.0,267.0,80.0,163.0,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY


In [5]:
nb_champions = 166

# Encode the input champion ids as one-hot for every champion
def encode_champion(champion_id):
  correspondances = [
    266, 103, 84, 166, 12, 32, 34, 1, 523, 22,
    136, 268, 432, 200, 53, 63, 201, 233, 51, 164,
    69, 31, 42, 122, 131, 119, 36, 245, 60, 28, 81,
    9, 114, 105, 3, 41, 86, 150, 79, 104, 887, 120,
    74, 910, 420, 39, 427, 40, 59, 24, 126, 202, 222,
    145, 429, 43, 30, 38, 55, 10, 141, 85, 121, 203,
    240, 96, 897, 7, 64, 89, 876, 127, 236, 117, 99,
    54, 90, 57, 11, 902, 21, 62, 82, 25, 950, 267,
    75, 111, 518, 76, 895, 56, 20, 2, 61, 516, 80,
    78, 555, 246, 133, 497, 33, 421, 526, 888, 58, 107,
    92, 68, 13, 360, 113, 235, 147, 875, 35, 98, 102,
    27, 14, 15, 72, 37, 16, 50, 517, 134, 223, 163,
    91, 44, 17, 412, 18, 48, 23, 4, 29, 77, 6, 110,
    67, 45, 161, 711, 254, 234, 112, 8, 106, 19, 498,
    101, 5, 157, 777, 83, 350, 154, 238, 221, 115, 26,
    142, 143
  ]
  encoding = np.zeros(nb_champions)
  index = correspondances.index(champion_id)
  encoding[index] = 1
  return encoding

def encode_champions(champions_ids):
  """
  Encodes the given list of champions ids into an input vector of size nb_champions * 10 (number of players in each LoL game)
  """
  encoding = np.array([])
  for id in champions_ids:
    encoding = np.concatenate((encoding, encode_champion(id)))
  return encoding

def encode_role(role):
  roles = {
      "TOP": 0,
      "JUNGLE": 1,
      "MIDDLE": 2,
      "BOTTOM": 3,
      "UTILITY": 4
  }
  role = role.strip()
  encoding = np.zeros(len(roles))
  encoding[roles[role]] = 1
  return encoding

def encode_roles(roles):
  encoding = np.array([])
  for role in roles:
    encoding = np.concatenate((encoding, encode_role(role)))
  return encoding

ids = [266, 76, 895, 56, 20, 2, 61, 516, 80, 78]
encode_champions(ids)
roles = ["TOP",	"JUNGLE",	"MIDDLE",	"BOTTOM",	"UTILITY",	"TOP",	"JUNGLE",	"MIDDLE",	"BOTTOM",	"UTILITY"]
encode_roles(roles)

array([1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.])

In [6]:
def augment_row(row):
  row = [int(champion_id) for champion_id in row[:10]] + [role for role in row[10:]]
  version_1 = np.concatenate((encode_champions(row[:10]), encode_roles(row[10:])))
  teams_reversed = row[5:10] + row[:5]
  roles_reversed = row[15:] + row[10:15]
  version_2 = np.concatenate((encode_champions(teams_reversed), encode_roles(roles_reversed)))
  return np.array(version_1), np.array(version_2)

def augment_data(X, y):
  nb_features = 1710
  new_X = np.array([augment_row(row) for row in X]).reshape((-1, nb_features))
  new_y = np.array([(label, 1-label) for label in y]).flatten()
  return new_X, new_y

In [7]:
# Split training, validation and test data
from sklearn.model_selection import train_test_split

X = df.drop(["WIN"], axis=1)
X = X.values

Y = df['WIN']
X, Y = augment_data(X, Y)
X.shape, Y.shape

((122534, 1710), (122534,))

In [8]:
X_train, X_, y_train, y_ = train_test_split(X, Y, test_size=0.15, stratify=Y)
X_val, X_test, y_val, y_test = train_test_split(X_, y_, test_size=0.5, stratify=y_)

y_train.shape, y_val.shape, y_test.shape, X_train.shape, X_val.shape, X_test.shape

((104153,), (9190,), (9191,), (104153, 1710), (9190, 1710), (9191, 1710))

In [9]:
from sklearn.metrics import accuracy_score, classification_report

def fit_eval_model(model, X_train, y_train, X_val, y_val,):
  model.fit(X_train, y_train)

  train_predictions = model.predict(X_train)
  train_accuracy = accuracy_score(y_train, train_predictions)

  eval_predictions = model.predict(X_val)
  eval_accuracy = accuracy_score(y_val, eval_predictions)

  print("Training Accuracy:", train_accuracy)
  print("Classification Report (Training):\n", classification_report(y_train, train_predictions))

  print("Evaluation Accuracy:", eval_accuracy)
  print("Classification Report (Evaluation):\n", classification_report(y_val, eval_predictions))
  return train_accuracy, eval_accuracy

In [ ]:
# First model: SVC
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

lambdas = [1 ** -i for i in range(5)]
degrees = list(range(1, 5))

train_accuracies, eval_accuracies = [[]] * len(lambdas), [[]] * len(lambdas)
"""
for idx_l, _lambda in enumerate(lambdas):
  for degree in  tqdm(degrees):
    model_svc = SVC(kernel='poly', C=_lambda, degree=degree) # With fewer examples, a linear kernel works best (0.6 val accuracy vs)
    model_svc.fit(X_train, y_train)

    train_predictions = model_svc.predict(X_train)
    train_accuracy = accuracy_score(y_train, train_predictions)
    train_accuracies[idx_l].append(train_accuracy)

    print("Training Accuracy:", train_accuracy)
    print("Classification Report (Training):\n", classification_report(y_train, train_predictions))

    eval_predictions = model_svc.predict(X_val)
    eval_accuracy = accuracy_score(y_val, eval_predictions)
    eval_accuracies[idx_l].append(eval_accuracy)

    print("Evaluation Accuracy:", eval_accuracy)
    print("Classification Report (Evaluation):\n", classification_report(y_val, eval_predictions))

train_accuracies, eval_accuracies"""
svc_model = SVC(kernel='poly')
svc_model.fit(X_train, y_train)

train_predictions = svc_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)

print("Training Accuracy:", train_accuracy)
print("Classification Report (Training):\n", classification_report(y_train, train_predictions))

eval_predictions = svc_model.predict(X_val)
eval_accuracy = accuracy_score(y_val, eval_predictions)

print("Evaluation Accuracy:", eval_accuracy)
print("Classification Report (Evaluation):\n", classification_report(y_val, eval_predictions))

In [ ]:
# Second model: Logistic regression
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

train_predictions = logistic_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)

eval_predictions = logistic_model.predict(X_val)
eval_accuracy = accuracy_score(y_val, eval_predictions)

print("Training Accuracy:", train_accuracy)
print("Classification Report (Training):\n", classification_report(y_train, train_predictions))

print("Evaluation Accuracy:", eval_accuracy)
print("Classification Report (Evaluation):\n", classification_report(y_val, eval_predictions))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training Accuracy: 0.5917782026768642
Classification Report (Training):
               precision    recall  f1-score   support

           0       0.59      0.57      0.58     21149
           1       0.59      0.62      0.60     21737

    accuracy                           0.59     42886
   macro avg       0.59      0.59      0.59     42886
weighted avg       0.59      0.59      0.59     42886

Evaluation Accuracy: 0.5337323177366703
Classification Report (Evaluation):
               precision    recall  f1-score   support

           0       0.53      0.51      0.52      5439
           1       0.54      0.56      0.55      5589

    accuracy                           0.53     11028
   macro avg       0.53      0.53      0.53     11028
weighted avg       0.53      0.53      0.53     11028



In [ ]:
# Third model: Decision trees
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)

train_predictions = tree_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)

eval_predictions = tree_model.predict(X_val)
eval_accuracy = accuracy_score(y_val, eval_predictions)

print("Training Accuracy:", train_accuracy)
print("Classification Report (Training):\n", classification_report(y_train, train_predictions))

print("Evaluation Accuracy:", eval_accuracy)
print("Classification Report (Evaluation):\n", classification_report(y_val, eval_predictions))

Training Accuracy: 1.0
Classification Report (Training):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     21149
           1       1.00      1.00      1.00     21737

    accuracy                           1.00     42886
   macro avg       1.00      1.00      1.00     42886
weighted avg       1.00      1.00      1.00     42886

Evaluation Accuracy: 0.5858723249909322
Classification Report (Evaluation):
               precision    recall  f1-score   support

           0       0.58      0.58      0.58      5439
           1       0.59      0.60      0.59      5589

    accuracy                           0.59     11028
   macro avg       0.59      0.59      0.59     11028
weighted avg       0.59      0.59      0.59     11028



In [15]:
# Fourth model: Random forests
from sklearn.ensemble import RandomForestClassifier

forest_model = RandomForestClassifier()
forest_model.fit(X_train, y_train)

train_predictions = forest_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)

eval_predictions = forest_model.predict(X_val)
eval_accuracy = accuracy_score(y_val, eval_predictions)

print("Training Accuracy:", train_accuracy)
print("Classification Report (Training):\n", classification_report(y_train, train_predictions))

print("Evaluation Accuracy:", eval_accuracy)
print("Classification Report (Evaluation):\n", classification_report(y_val, eval_predictions))

Training Accuracy: 1.0
Classification Report (Training):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     52076
           1       1.00      1.00      1.00     52077

    accuracy                           1.00    104153
   macro avg       1.00      1.00      1.00    104153
weighted avg       1.00      1.00      1.00    104153

Evaluation Accuracy: 0.6078346028291621
Classification Report (Evaluation):
               precision    recall  f1-score   support

           0       0.60      0.63      0.62      4595
           1       0.61      0.59      0.60      4595

    accuracy                           0.61      9190
   macro avg       0.61      0.61      0.61      9190
weighted avg       0.61      0.61      0.61      9190



In [ ]:
# Fifth model: XGBoost
from xgboost import XGBClassifier

xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

train_predictions = xgb_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)

eval_predictions = xgb_model.predict(X_val)
eval_accuracy = accuracy_score(y_val, eval_predictions)

print("Training Accuracy:", train_accuracy)
print("Classification Report (Training):\n", classification_report(y_train, train_predictions))

print("Evaluation Accuracy:", eval_accuracy)
print("Classification Report (Evaluation):\n", classification_report(y_val, eval_predictions))

Training Accuracy: 0.6496758849041645
Classification Report (Training):
               precision    recall  f1-score   support

           0       0.66      0.59      0.63     21149
           1       0.64      0.71      0.67     21737

    accuracy                           0.65     42886
   macro avg       0.65      0.65      0.65     42886
weighted avg       0.65      0.65      0.65     42886

Evaluation Accuracy: 0.5428001450852376
Classification Report (Evaluation):
               precision    recall  f1-score   support

           0       0.54      0.49      0.51      5439
           1       0.54      0.60      0.57      5589

    accuracy                           0.54     11028
   macro avg       0.54      0.54      0.54     11028
weighted avg       0.54      0.54      0.54     11028



In [ ]:
# Sixth model: Naive Bayes
from sklearn.naive_bayes import BernoulliNB

nb_model = BernoulliNB()
nb_model.fit(X_train, y_train)

train_predictions = nb_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)

eval_predictions = nb_model.predict(X_val)
eval_accuracy = accuracy_score(y_val, eval_predictions)

print("Training Accuracy:", train_accuracy)
print("Classification Report (Training):\n", classification_report(y_train, train_predictions))

print("Evaluation Accuracy:", eval_accuracy)
print("Classification Report (Evaluation):\n", classification_report(y_val, eval_predictions))

Training Accuracy: 0.5853658536585366
Classification Report (Training):
               precision    recall  f1-score   support

           0       0.58      0.59      0.58     21149
           1       0.59      0.58      0.59     21737

    accuracy                           0.59     42886
   macro avg       0.59      0.59      0.59     42886
weighted avg       0.59      0.59      0.59     42886

Evaluation Accuracy: 0.5349111352919841
Classification Report (Evaluation):
               precision    recall  f1-score   support

           0       0.53      0.54      0.53      5439
           1       0.54      0.53      0.54      5589

    accuracy                           0.53     11028
   macro avg       0.53      0.53      0.53     11028
weighted avg       0.54      0.53      0.53     11028



In [11]:
# Seventh model: MLP
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

nb_champions = 166
nb_roles = 5
nb_players = 10
input_dim = (nb_champions + nb_roles) * nb_players

# A simple Multi-Layer Perceptron model with four layers: input layer with 64 ReLU units, and two hidden layers with ReLU units (32 and 16 respectively).
# Output layer uses a sigmoid activation function since we're doing binary classification
mlp_model = Sequential([
    Dense(units=64, activation='relu', input_dim=input_dim),
    Dense(units=32, activation='relu'),
    Dense(units=16, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

mlp_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
X_train, X_val = np.array(X_train), np.array(X_val)
history = mlp_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/50
3255/3255 [==============================] - 17s 4ms/step - loss: 0.6908 - accuracy: 0.5247 - val_loss: 0.6885 - val_accuracy: 0.5311
Epoch 2/50
3255/3255 [==============================] - 14s 4ms/step - loss: 0.6791 - accuracy: 0.5620 - val_loss: 0.6875 - val_accuracy: 0.5508
Epoch 3/50
3255/3255 [==============================] - 14s 4ms/step - loss: 0.6442 - accuracy: 0.6131 - val_loss: 0.7048 - val_accuracy: 0.5428
Epoch 4/50
3255/3255 [==============================] - 14s 4ms/step - loss: 0.5812 - accuracy: 0.6740 - val_loss: 0.7565 - val_accuracy: 0.5561
Epoch 5/50
3255/3255 [==============================] - 13s 4ms/step - loss: 0.5132 - accuracy: 0.7290 - val_loss: 0.8224 - val_accuracy: 0.5624
Epoch 6/50
3255/3255 [==============================] - 13s 4ms/step - loss: 0.4586 - accuracy: 0.7649 - val_loss: 0.9129 - val_accuracy: 0.5653
Epoch 7/50
3255/3255 [==============================] - 13s 4ms/step - loss: 0.4151 - accuracy: 0.7905 - val_loss: 0.9996 - val_ac

In [12]:
mlp_model.save('mlp-model.keras')

loss_mlp, accuracy_mlp = mlp_model.evaluate(X_test, y_test)
loss_mlp, accuracy_mlp

288/288 [==============================] - 1s 2ms/step - loss: 5.1663 - accuracy: 0.5981


(5.166328430175781, 0.5980851054191589)

In [13]:
# Eighth model: RNN using LSTM units
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

nb_champions = 166
nb_roles = 5
nb_players = 10
input_dim = (nb_champions + nb_roles) * nb_players

embedding_dim=256
X_train_rnn, X_val_rnn, X_test_rnn = X_train.reshape((X_train.shape[0], 1, input_dim)), X_val.reshape((X_val.shape[0], 1, input_dim)), X_test.reshape((X_test.shape[0], 1, input_dim))
y_train_rnn, y_val_rnn, y_test_rnn = y_train.reshape((-1, 1)), y_val.reshape((-1, 1)), y_test.reshape((-1, 1))
print(X_train_rnn.shape)
print(X_train_rnn)

rnn_model = Sequential([
    LSTM(units=40, input_shape=(1, input_dim), return_sequences=True),
    LSTM(units=20, return_sequences=True),
    Dense(units=1, activation='sigmoid')
])

rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

rnn_history = rnn_model.fit(X_train_rnn, y_train_rnn, epochs=50, batch_size=32, validation_data=(X_val_rnn, y_val_rnn))

(104153, 1, 1710)
[[[0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 1.]]

 ...

 [[0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 1.]]]
Epoch 1/50
3255/3255 [==============================] - 29s 7ms/step - loss: 0.6906 - accuracy: 0.5250 - val_loss: 0.6889 - val_accuracy: 0.5329
Epoch 2/50
3255/3255 [==============================] - 21s 6ms/step - loss: 0.6821 - accuracy: 0.5551 - val_loss: 0.6902 - val_accuracy: 0.5326
Epoch 3/50
3255/3255 [==============================] - 22s 7ms/step - loss: 0.6702 - accuracy: 0.5743 - val_loss: 0.6962 - val_accuracy: 0.5349
Epoch 4/50
3255/3255 [==============================] - 35s 11ms/step - loss: 0.6389 - accuracy: 0.6184 - val_loss: 0.7265 - val_accuracy: 0.5428
Epoch 5/50
3255/3255 [==============================] - 22s 7ms/step - loss: 0.5658 - accuracy: 0.6889 - val_loss: 0.7904 - val_accuracy: 0.5569
Epoch 6/50
3255/3255 [==============================] - 22s 7ms/step - loss: 0.476

In [14]:
rnn_model.save('rnn-model.keras')

loss_rnn, accuracy_rnn = rnn_model.evaluate(X_test_rnn, y_test_rnn)
loss_rnn, accuracy_rnn

288/288 [==============================] - 1s 4ms/step - loss: 4.1240 - accuracy: 0.5973


(4.124024391174316, 0.597323477268219)

In [19]:
# Ninth model: RNNs using LSTM layers with Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout

nb_champions = 166
nb_roles = 5
nb_players = 10
input_dim = (nb_champions + nb_roles) * nb_players

X_train_rnn, X_val_rnn, X_test_rnn = X_train.reshape((X_train.shape[0], 1, input_dim)), X_val.reshape((X_val.shape[0], 1, input_dim)), X_test.reshape((X_test.shape[0], 1, input_dim))
y_train_rnn, y_val_rnn, y_test_rnn = y_train.reshape((-1, 1)), y_val.reshape((-1, 1)), y_test.reshape((-1, 1))

# A second RNN model using LSTM layers with dropout to see if it reduces variance
rnn_model2 = Sequential([
    LSTM(units=40, input_shape=(1, input_dim),return_sequences=True),
    Dropout(0.2),
    LSTM(units=20, return_sequences=True),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])

rnn_model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

rnn_history2 = rnn_model2.fit(X_train_rnn, y_train_rnn, epochs=50, batch_size=32, validation_data=(X_val_rnn, y_val_rnn))

Epoch 1/50
3255/3255 [==============================] - 21s 6ms/step - loss: 0.6906 - accuracy: 0.5253 - val_loss: 0.6889 - val_accuracy: 0.5320
Epoch 2/50
3255/3255 [==============================] - 18s 5ms/step - loss: 0.6830 - accuracy: 0.5531 - val_loss: 0.6893 - val_accuracy: 0.5317
Epoch 3/50
3255/3255 [==============================] - 19s 6ms/step - loss: 0.6750 - accuracy: 0.5691 - val_loss: 0.6900 - val_accuracy: 0.5405
Epoch 4/50
3255/3255 [==============================] - 18s 6ms/step - loss: 0.6575 - accuracy: 0.5975 - val_loss: 0.6995 - val_accuracy: 0.5493
Epoch 5/50
3255/3255 [==============================] - 17s 5ms/step - loss: 0.6170 - accuracy: 0.6469 - val_loss: 0.7254 - val_accuracy: 0.5569
Epoch 6/50
3255/3255 [==============================] - 17s 5ms/step - loss: 0.5632 - accuracy: 0.6935 - val_loss: 0.7584 - val_accuracy: 0.5588
Epoch 7/50
3255/3255 [==============================] - 19s 6ms/step - loss: 0.5128 - accuracy: 0.7316 - val_loss: 0.8125 - val_ac

In [21]:
rnn_model2.save('rnn-model2.keras')

loss2, acc2 = rnn_model2.evaluate(X_test_rnn, y_test_rnn)
loss2, acc2
x = np.array()
rnn_model2.predict

288/288 [==============================] - 1s 3ms/step - loss: 1.5260 - accuracy: 0.5984


(1.5260289907455444, 0.598411500453949)

In [ ]:
import numpy as np

def make_predictions(X_data):
    roles = ["TOP",	"JUNGLE",	"MIDDLE",	"BOTTOM",	"UTILITY",	"TOP",	"JUNGLE",	"MIDDLE",	"BOTTOM",	"UTILITY"]
    champion_encoding = encode_champions(X_data)
    roles_encoding = encode_roles(roles)

    # Concatenate the two encodings along the second axis (axis=1)
    X = np.concatenate((champion_encoding, roles_encoding)).reshape((1, -1))

    # Make predictions using the trained model
    predictions = mlp_model.predict(X)
    threshold=0.5
    # If your output layer uses sigmoid activation for binary classification, you might want to threshold the predictions
    binary_predictions = (predictions > threshold).astype(int)

    return binary_predictions

# Example usage:
predictions = make_predictions()
print(predictions)

predictions2 = make_predictions()
print(predictions2)